In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import imutils
from PIL import Image
import keras
from keras import Sequential, losses, optimizers
from keras.layers import Dense, Conv2D, MaxPool2D, Average, Flatten, Dropout, Activation
from keras.utils import to_categorical, plot_model,vis_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard, EarlyStopping
from keras.preprocessing.image import img_to_array
from skimage import io
import imutils
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab import files
from google.colab import drive
import glob
import pywt
from scipy import signal
import ecg_plot
drive.mount('/content/gdrive')

Using TensorFlow backend.


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
pip install ecg_plot

In [0]:
class DataModel:
    """
            DataModel
            not used currently in the program.
    """
    def __init__(self):
        self.items_labels = None
        self.images = []
        self.curated_data = {}
    def add_image(self, image_array):
        self.images.append(image_array)
    def add_labels(self, df):
        self.item_labels = df
    def add_curated_data(self, image, emotion):
        self.curated_data['segment'] = image #Array from csv of each segment
        self.curated_data['label'] = label #Label
        self.curated_data['scalogram'] =scaledImage #scalogram of each segment

def load_data(number_of_items=20):
    """
        number_of_items -> Number of items to return
        returns the data in a dictionary of images and labels.
    """
    path = "gdrive/My Drive/CMPE295B/csv_segmented_individual"
    data = [] 
    curated_data = {"segments":[], "label":[], "scalogram":[]}
    for subject_name in os.listdir(path)[:number_of_items]:
        # At the start of the iteration build a data model
        #data_model = DataModel()
        if subject_name == ".DS_Store":
            continue
        print ("Going through subject:" + subject_name)
        base=os.path.basename(path+"/"+subject_name)
        labelData=os.path.splitext(base)[0]
        print(labelData)
        i=1
        for items in os.listdir(path+"/"+subject_name):
          #print (items)
          print(i)
          if items == ".DS_Store":
            continue
          else:
            try:
              if items.endswith(".csv"):
                i=i+1
                #--------Approach 1------------------
                df1=np.genfromtxt(path+"/"+subject_name+"/"+items, delimiter=',')
                #print(type(df1))
                print(df1.shape)
                #-----------------------To show Segment plot. Comment scalogram plt---
                plt.plot(df1)
                plt.grid(True)
                plt.savefig(path+"/"+subject_name+"/"+"plotECG.png")
                plt.show() 
                image1=cv2.imread(path+"/"+subject_name+"/"+"plotECG.png")
                image1=cv2.resize(image1,(250,250))
                #plt.imshow(image1)
                #plt.show()
                #print(image1.shape)
                #---------------------------------------------------------
                """df1=np.genfromtxt(path+"/"+subject_name+"/"+items, delimiter=',')
                #print(type(df1))
                #-----------------------To show Segment plot. Comment scalogram plt---
                #plt.plot(df1)
                #plt.show()"""

              #curated_data['segments'].append(df1)
              curated_data['scalogram'].append(image1)
              curated_data['label'].append(labelData)
            
            except:
              df = None                    
        #data.append(data_model) # Save all the data

    return curated_data


In [0]:
data = load_data(number_of_items=12)

In [0]:
print(len(data['scalogram']))
print(data['label'].shape)
encoder = LabelEncoder()
encoder.fit(data['label'])

6877


LabelEncoder()

In [0]:
# Label Encoding all the emotion data
data['label'] = encoder.transform(data['label'])

In [0]:
all_the_classes = encoder.classes_
mapping = {0: 'person100',
 1: 'person101',
 2: 'person102',
 3: 'person103',
 4: 'person104',
 5: 'person105',
 6: 'person106',
 7: 'person107',
 8: 'person108',
 9: 'person109',
 10: 'person111',
 11: 'person112',
 12: 'person113',
 13: 'person114',
 14: 'person115',
 15: 'person116',
 16: 'person117',
 17: 'person118',}

# Get emotion from class number   
def get_emotion_from_class(class_number):
    """
        gets the corresponding emotion from the class
    """
    if mapping.get(class_number,None):
        return mapping.get(class_number)
    else:
        return -1 # No such class

In [0]:
# Extracting features and labels
features = np.array(data['scalogram'])
labels = data['label']

In [0]:
print ("Shape of features: ", features.shape)
print ("Shape of labels: ", labels.shape)

Shape of features:  (6877, 250, 250, 3)
Shape of labels:  (6877,)


In [0]:
labels = to_categorical(labels, num_classes=len(mapping))

In [0]:
model = Sequential()
model.add(Conv2D(8, kernel_size=5, input_shape=(250,250,3), activation="relu"))
model.add(Conv2D(8, kernel_size=5, activation="relu"))
model.add(MaxPool2D(2))
model.add(Dropout(0.25))
#model.add(Conv2D(16, kernel_size=5, activation="relu"))
#model.add(Conv2D(16, kernel_size=5, activation="relu"))
#model.add(MaxPool2D(2))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(50, activation="relu"))
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(50, activation="relu"))
model.add(Dense(len(mapping), activation="softmax"))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 246, 246, 8)       608       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 242, 242, 8)       1608      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 121, 121, 8)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 121, 121, 8)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 117, 117, 16)      3216      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 113, 113, 16)      6416      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 16)       

In [0]:

model.compile(optimizer=optimizers.adam(lr=0.001), loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [0]:
# Splitting
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, shuffle=True, random_state=34)
print(labels_train.shape)
print(features_train.shape)

(5157, 18)
(5157, 250, 250, 3)


In [0]:
tensorboard = TensorBoard()
earlystopping = EarlyStopping(patience=3)

In [0]:
features_train = features_train.astype("float32")
features_test = features_test.astype("float32")

In [0]:

features_train = features_train / 1/255
features_test = features_test / 1/255

print(features_train.shape)
print(features_test.shape)

(5157, 250, 250, 3)
(1720, 250, 250, 3)


In [0]:
features_train = features_train.reshape(len(features_train), 250, 250, 3)
features_test = features_test.reshape(len(features_test), 250, 250, 3)
print(features_train.shape)
print(features_test.shape)
labels_train1=[]

(5157, 250, 250, 3)
(1720, 250, 250, 3)


In [0]:
# Training
history = model.fit(features_train, labels_train, epochs=100, batch_size=32, callbacks=[tensorboard,earlystopping], validation_data=(features_test, labels_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 5157 samples, validate on 1720 samples







Epoch 1/100
5157/5157 [==============================] - 14s 3ms/step - loss: 1.4211 - acc: 0.4590 - val_loss: 0.1102 - val_acc: 0.9674

Epoch 2/100
5157/5157 [==============================] - 7s 1ms/step - loss: 0.1061 - acc: 0.9659 - val_loss: 0.0669 - val_acc: 0.9791
Epoch 3/100
5157/5157 [==============================] - 7s 1ms/step - loss: 0.0673 - acc: 0.9822 - val_loss: 0.0452 - val_acc: 0.9831
Epoch 4/100
5157/5157 [==============================] - 7s 1ms/step - loss: 0.0474 - acc: 0.9872 - val_loss: 0.0305 - val_acc: 0.9901
Epoch 5/100
5157/5157 [==============================] - 7s 1ms/step - loss: 0.0337 - acc: 0.9915 - val_loss: 0.0528 - val_acc: 0.9878
Epoch 6/100
5157/5157 [==============================] - 7s 1ms/step - loss: 0.0233 - acc: 0.9936 - val_loss: 0.0289 - val_acc: 0.9919
Epoch 7/100
5157/5157 [=============